In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import sidetable
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

In [2]:
df_da = pd.read_csv('data/df_da.csv', index_col = 'index')
pd.options.display.max_columns = None
df_da.sample(4)

,age,gender,country,education,role,prog_language,code_env,notebook_kaggle,notebook_colab,notebook_azure,notebook_paperspace,notebook_binder,notebook_codeocean,notebook_ibmwatsonstudio,notebook_amazonsagemaker,notebook_amazonemr,notebook_googlecloudnotebooks,notebook_googleclouddatalab,notebook_databricks,notebook_zeppelin,notebook_deepnote,notebook_observable,notebook_OTHER,viz,field,salary,learn_dp_MySQL,learn_dp_PostgreSQL,learn_dp_SQLite,learn_dp_Oracle,learn_dp_Mongo,learn_dp_Snowflake,learn_dp_IBMDd2,learn_dp_SQLServer,learn_dp_AzureSQL,learn_dp_AzureCosmos,learn_dp_Redshift,learn_dp_Aurora,learn_dp_RDS,learn_dp_DynamoDB,learn_dp_BigQuery,learn_dp_CloudSQL,learn_dp_Firestore,learn_dp_BigTable,learn_dp_Spanner,learn_dp_Other,dp_most_freq,BI_tools,share_plotlydash,share_streamlit,share_NBViewer,share_GitHub,share_Personal blog,share_Kaggle,share_Colab,share_Shiny,share_None,share_OTHER,courses_Coursera,courses_edX,courses_Kaggle,courses_DataCamp,courses_Fastai,courses_Udacity,courses_Udemy,courses_LinkedIn,courses_Cloud-certification,courses_University,courses_OTHER,media_Twitter,media_Email,media_Reddit,media_Kaggle,media_Forums,media_YouTube,media_Podcasts,media_Blogs,media_Journal,media_Slack,media_OTHER,cat_experience,cat_salary
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
16566,22-24,Man,Other,Bachelor’s degree,Data Analyst,None,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Online Service/Internet-based Services,$0-999,NaN,PostgreSQL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I do not share my work publicly,NaN,NaN,edX,Kaggle Learn Courses,DataCamp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Kaggle (notebooks, forums, etc)",NaN,"YouTube (Kaggle YouTube, Cloud AI Adventures, ...",NaN,NaN,NaN,NaN,NaN,Mid,< 25K
12978,25-29,Woman,Austria,Some college/university study without earning ...,Data Analyst,Python,Visual Studio Code (VSCode),NaN,NaN,NaN,NaN,Binder / JupyterHub,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Matplotlib ,Seaborn ,Plotly / Plotly Express",Computers/Technology,"15,000-19,999",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Junior,< 25K
19468,30-34,Woman,Malaysia,Master’s degree,Data Analyst,"Python, SQL, C, Other","JupyterLab , Notepad++,Jupyter Notebook",Kaggle Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Matplotlib ,Seaborn",Manufacturing/Fabrication,"10,000-14,999",MySQL,PostgreSQL,NaN,Oracle Database,MongoDB,NaN,NaN,Microsoft SQL Server,Microsoft Azure SQL Database,NaN,NaN,NaN,NaN,NaN,Google Cloud BigQuery,Google Cloud SQL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kaggle Learn Courses,NaN,NaN,NaN,Udemy,LinkedIn Learning,NaN,NaN,Other,Twitter (data science influencers),NaN,NaN,"Kaggle (notebooks, forums, etc)",NaN,"YouTube (Kaggle YouTube, Cloud AI Adventures, ...","Podcasts (Chai Time Data Science, O’Reilly Dat...","Blogs (Towards Data Science, Analytics Vidhya,...",NaN,NaN,NaN,Senior,< 25K
24487,35-39,Man,Singapore,Master’s degree,Business Analyst,Other,Visual Studio,Kaggle Notebooks,NaN,NaN,NaN,Binder / JupyterHub,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Senior+,NaN


In [3]:
df_da.describe(include='object').T

,count,unique,top,freq
age,3269,11,25-29,788
gender,3269,5,Man,2538
country,3269,66,India,795
education,3269,7,Master’s degree,1514
role,3269,2,Data Analyst,2301
...,...,...,...,...
media_Journal,260,1,"Journal Publications (peer-reviewed journals, ...",260
media_Slack,258,1,"Slack Communities (ods.ai, kagglenoobs, etc)",258
media_OTHER,82,1,Other,82
cat_experience,3269,5,Junior,1881


In [4]:
df_da.stb.missing()

,missing,total,percent
notebook_codeocean,3247,3269,99.327011
notebook_paperspace,3246,3269,99.296421
notebook_deepnote,3243,3269,99.204650
notebook_zeppelin,3241,3269,99.143469
share_NBViewer,3241,3269,99.143469
...,...,...,...
role,0,3269,0.000000
education,0,3269,0.000000
country,0,3269,0.000000
cat_experience,0,3269,0.000000


In [5]:
# creamos diferentes df con datos sociodemográficos propios para cada columna que nos interesa 
# y poder expandirlas sin crear demasiadas filas


sociodemo_language = df_da[["age","gender", "country", "education", "role", "field", "cat_experience", "cat_salary","prog_language"] ]
sociodemo_language["languages"] = sociodemo_language["prog_language"].str.split(",")
df_languages = sociodemo_language.explode(["languages"])
df_languages.drop("prog_language", inplace= True, axis= 1)
df_languages.sample(2)

/tmp/ipykernel_19691/4121504353.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sociodemo_language["languages"] = sociodemo_language["prog_language"].str.split(",")


,age,gender,country,education,role,field,cat_experience,cat_salary,languages
index,,,,,,,,,
25488,45-49,Man,Ukraine,Master’s degree,Data Analyst,Online Business/Internet-based Sales,Senior,< 25K,SQL
23009,35-39,Man,France,Master’s degree,Data Analyst,Academics/Education,Mid,< 25K,Python


In [6]:
sociodemo_code_env = df_da[["age","gender", "country", "education", "role", "field", "cat_experience", "cat_salary","code_env"] ]
sociodemo_code_env["cod_env"] = sociodemo_code_env["code_env"].str.split(",")
df_code_env = sociodemo_code_env.explode(["cod_env"])
df_code_env.drop("code_env", inplace= True, axis= 1)
df_code_env.sample(2)

/tmp/ipykernel_19691/993416549.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sociodemo_code_env["cod_env"] = sociodemo_code_env["code_env"].str.split(",")


,age,gender,country,education,role,field,cat_experience,cat_salary,cod_env
index,,,,,,,,,
5922,45-49,Man,Spain,Doctoral degree,Business Analyst,Energy/Mining,Senior+,50K - 100K,PyCharm
5864,40-44,Man,Mexico,Doctoral degree,Business Analyst,Accounting/Finance,Senior,NaN,Jupyter Notebook


In [7]:
sociodemo_viz = df_da[["age","gender", "country", "education", "role", "field", "cat_experience", "cat_salary","viz"] ]
sociodemo_viz["visual"] = sociodemo_viz["viz"].str.split(",")
df_viz = sociodemo_viz.explode(["visual"])
df_viz.drop("viz", inplace= True, axis= 1)
df_viz.sample(2)

/tmp/ipykernel_19691/2257016201.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sociodemo_viz["visual"] = sociodemo_viz["viz"].str.split(",")


,age,gender,country,education,role,field,cat_experience,cat_salary,visual
index,,,,,,,,,
14213,30-34,Prefer not to say,United States of America,Master’s degree,Data Analyst,Manufacturing/Fabrication,Mid,NaN,Matplotlib
14491,35-39,Man,Ireland,Some college/university study without earning ...,Data Analyst,Broadcasting/Communications,Senior+,50K - 100K,Matplotlib


In [8]:
#Creamos la función para crear una columna que una las anteriores:
def unir(col):
    lista_valores = []
    for element in col:
        if str(element) != "nan":
            lista_valores.append(str(element))
        else:
            pass
    return lista_valores

In [9]:
# creamos una lista con todas las columnas relacionadas a la pregunta notebooks

lista_notebooks = ['notebook_kaggle', 'notebook_colab', 'notebook_azure',
       'notebook_paperspace', 'notebook_binder', 'notebook_codeocean',
       'notebook_ibmwatsonstudio', 'notebook_amazonsagemaker',
       'notebook_amazonemr', 'notebook_googlecloudnotebooks',
       'notebook_googleclouddatalab', 'notebook_databricks',
       'notebook_zeppelin', 'notebook_deepnote', 'notebook_observable',
       'notebook_OTHER']

In [10]:
# aplicamos la función de unir columnas sobre el df completo pero uniendo solo las columnas 'notebooks':
df_da['notebooks_unido'] = df_da[lista_notebooks].apply(unir, axis=1)

In [11]:
#comprobamos:
df_da.head(2)

,age,gender,country,education,role,prog_language,code_env,notebook_kaggle,notebook_colab,notebook_azure,notebook_paperspace,notebook_binder,notebook_codeocean,notebook_ibmwatsonstudio,notebook_amazonsagemaker,notebook_amazonemr,notebook_googlecloudnotebooks,notebook_googleclouddatalab,notebook_databricks,notebook_zeppelin,notebook_deepnote,notebook_observable,notebook_OTHER,viz,field,salary,learn_dp_MySQL,learn_dp_PostgreSQL,learn_dp_SQLite,learn_dp_Oracle,learn_dp_Mongo,learn_dp_Snowflake,learn_dp_IBMDd2,learn_dp_SQLServer,learn_dp_AzureSQL,learn_dp_AzureCosmos,learn_dp_Redshift,learn_dp_Aurora,learn_dp_RDS,learn_dp_DynamoDB,learn_dp_BigQuery,learn_dp_CloudSQL,learn_dp_Firestore,learn_dp_BigTable,learn_dp_Spanner,learn_dp_Other,dp_most_freq,BI_tools,share_plotlydash,share_streamlit,share_NBViewer,share_GitHub,share_Personal blog,share_Kaggle,share_Colab,share_Shiny,share_None,share_OTHER,courses_Coursera,courses_edX,courses_Kaggle,courses_DataCamp,courses_Fastai,courses_Udacity,courses_Udemy,courses_LinkedIn,courses_Cloud-certification,courses_University,courses_OTHER,media_Twitter,media_Email,media_Reddit,media_Kaggle,media_Forums,media_YouTube,media_Podcasts,media_Blogs,media_Journal,media_Slack,media_OTHER,cat_experience,cat_salary,notebooks_unido
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,"Python, SQL","JupyterLab ,Jupyter Notebook",Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Databricks Collaborative Notebooks,NaN,NaN,NaN,NaN,"Matplotlib ,Seaborn ,Plotly / Plotly Express",Energy/Mining,"2,000-2,999",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Microsoft Azure SQL Database,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coursera,NaN,Kaggle Learn Courses,NaN,NaN,NaN,NaN,NaN,"Cloud-certification programs (direct from AWS,...",NaN,NaN,NaN,NaN,NaN,"Kaggle (notebooks, forums, etc)",NaN,NaN,NaN,"Blogs (Towards Data Science, Analytics Vidhya,...",NaN,NaN,NaN,Senior+,< 25K,"[ Kaggle Notebooks, Colab Notebooks, Databric..."
32,22-24,Nonbinary,United States of America,Some college/university study without earning ...,Data Analyst,R,RStudio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Google Cloud Datalab,NaN,NaN,NaN,NaN,NaN,Ggplot / ggplot2,Manufacturing/Fabrication,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Junior,NaN,[Google Cloud Datalab]


In [12]:
# Nos vamos a quedar solo con las columnas sociodemograficas y la nueva creada:

sociodemo_notebooks = df_da[["age","gender", "country", "education", "role", "field", "cat_experience", "cat_salary","notebooks_unido"]]
df_notebooks = sociodemo_notebooks.explode(["notebooks_unido"])
df_notebooks.head()

,age,gender,country,education,role,field,cat_experience,cat_salary,notebooks_unido
index,,,,,,,,,
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,Kaggle Notebooks
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,Colab Notebooks
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,Databricks Collaborative Notebooks
32,22-24,Nonbinary,United States of America,Some college/university study without earning ...,Data Analyst,Manufacturing/Fabrication,Junior,NaN,Google Cloud Datalab
33,30-34,Woman,Egypt,Bachelor’s degree,Data Analyst,Non-profit/Service,Mid,< 25K,Kaggle Notebooks


In [13]:
# vamos a comprobar si quedan nulos en esta columna (muestra que no haya usado nunca uno de estos notebooks):

df_notebooks['notebooks_unido'].isnull().sum()

1219

In [14]:
# vamos a eliminar estas filas (solo queremos saber los notebooks más utilizados):

df_notebooks.dropna(subset= 'notebooks_unido', axis = 0, inplace= True)

In [15]:
df_notebooks['notebooks_unido'].isnull().sum()

0

In [16]:
# vamos a hacer lo mismo con las columnas learn:

col_lear_dp = ['learn_dp_MySQL', 'learn_dp_PostgreSQL', 'learn_dp_SQLite', 'learn_dp_Oracle', 'learn_dp_Mongo', 'learn_dp_Snowflake', 'learn_dp_IBMDd2', 'learn_dp_SQLServer',
'learn_dp_AzureSQL', 'learn_dp_AzureCosmos', 'learn_dp_Redshift', 'learn_dp_Aurora',
'learn_dp_RDS', 'learn_dp_DynamoDB', 'learn_dp_BigQuery', 'learn_dp_CloudSQL', 'learn_dp_Firestore', 'learn_dp_BigTable', 'learn_dp_Spanner', 'learn_dp_Other']

In [17]:
df_da['learn_unido'] = df_da[col_lear_dp].apply(unir, axis=1)

In [18]:
sociodemo_learn = df_da[["age","gender", "country", "education", "role", "field", "cat_experience", "cat_salary","learn_unido"]]
df_learn = sociodemo_learn.explode(["learn_unido"])
df_learn.head()

,age,gender,country,education,role,field,cat_experience,cat_salary,learn_unido
index,,,,,,,,,
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,Microsoft Azure SQL Database
32,22-24,Nonbinary,United States of America,Some college/university study without earning ...,Data Analyst,Manufacturing/Fabrication,Junior,NaN,NaN
33,30-34,Woman,Egypt,Bachelor’s degree,Data Analyst,Non-profit/Service,Mid,< 25K,MySQL
33,30-34,Woman,Egypt,Bachelor’s degree,Data Analyst,Non-profit/Service,Mid,< 25K,SQLite
33,30-34,Woman,Egypt,Bachelor’s degree,Data Analyst,Non-profit/Service,Mid,< 25K,MongoDB


In [19]:
df_learn['learn_unido'].isnull().sum()

2134

In [20]:
df_learn.dropna(subset= 'learn_unido', axis = 0, inplace= True)

In [21]:
df_learn['learn_unido'].isnull().sum()

0

In [22]:
col_share = ['share_plotlydash', 'share_streamlit', 'share_NBViewer', 'share_GitHub', 'share_Personal blog', 'share_Kaggle', 'share_Colab', 'share_Shiny', 'share_None', 'share_OTHER']

In [23]:
df_da['share_unido'] = df_da[col_share].apply(unir, axis=1)

In [24]:
sociodemo_share = df_da[["age","gender", "country", "education", "role", "field", "cat_experience", "cat_salary","share_unido"]]
df_share = sociodemo_share.explode(["share_unido"])
df_share.head()

,age,gender,country,education,role,field,cat_experience,cat_salary,share_unido
index,,,,,,,,,
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,NaN
32,22-24,Nonbinary,United States of America,Some college/university study without earning ...,Data Analyst,Manufacturing/Fabrication,Junior,NaN,NaN
33,30-34,Woman,Egypt,Bachelor’s degree,Data Analyst,Non-profit/Service,Mid,< 25K,I do not share my work publicly
46,22-24,Man,China,Some college/university study without earning ...,Data Analyst,Computers/Technology,Junior,< 25K,NaN
52,40-44,Man,South Africa,Master’s degree,Data Analyst,Manufacturing/Fabrication,Junior,25K - 50K,I do not share my work publicly


In [25]:
df_share['share_unido'].isnull().sum()

1610

In [26]:
df_share.dropna(subset= 'share_unido', axis = 0, inplace= True)

In [27]:
df_share['share_unido'].isnull().sum()

0

In [28]:
col_courses = ['courses_Coursera', 'courses_edX', 'courses_Kaggle', 'courses_DataCamp', 'courses_Fastai', 'courses_Udacity', 'courses_Udemy', 'courses_LinkedIn',
'courses_Cloud-certification', 'courses_University', 'courses_OTHER']

In [29]:
df_da['courses_unido'] = df_da[col_courses].apply(unir, axis=1)

In [30]:
sociodemo_courses = df_da[["age","gender", "country", "education", "role", "field", "cat_experience", "cat_salary","courses_unido"]]
df_courses = sociodemo_courses.explode(["courses_unido"])
df_courses.head()

,age,gender,country,education,role,field,cat_experience,cat_salary,courses_unido
index,,,,,,,,,
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,Coursera
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,Kaggle Learn Courses
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,"Cloud-certification programs (direct from AWS,..."
32,22-24,Nonbinary,United States of America,Some college/university study without earning ...,Data Analyst,Manufacturing/Fabrication,Junior,NaN,NaN
33,30-34,Woman,Egypt,Bachelor’s degree,Data Analyst,Non-profit/Service,Mid,< 25K,Coursera


In [31]:
df_courses['courses_unido'].isnull().sum()

889

In [32]:
df_courses.dropna(subset= 'courses_unido', axis = 0, inplace= True)

In [33]:
df_courses['courses_unido'].isnull().sum()

0

In [34]:
col_media = ['media_Twitter', 'media_Email', 'media_Reddit', 'media_Kaggle', 'media_Forums', 'media_YouTube', 'media_Podcasts', 'media_Blogs', 'media_Journal', 'media_Slack', 'media_OTHER']

In [35]:
df_da['media_unido'] = df_da[col_media].apply(unir, axis=1)

In [36]:
sociodemo_media = df_da[["age","gender", "country", "education", "role", "field", "cat_experience", "cat_salary","media_unido"]]
df_media = sociodemo_media.explode(["media_unido"])
df_media.head()

,age,gender,country,education,role,field,cat_experience,cat_salary,media_unido
index,,,,,,,,,
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,"Kaggle (notebooks, forums, etc)"
16,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,Energy/Mining,Senior+,< 25K,"Blogs (Towards Data Science, Analytics Vidhya,..."
32,22-24,Nonbinary,United States of America,Some college/university study without earning ...,Data Analyst,Manufacturing/Fabrication,Junior,NaN,NaN
33,30-34,Woman,Egypt,Bachelor’s degree,Data Analyst,Non-profit/Service,Mid,< 25K,Twitter (data science influencers)
33,30-34,Woman,Egypt,Bachelor’s degree,Data Analyst,Non-profit/Service,Mid,< 25K,"Email newsletters (Data Elixir, O'Reilly Data ..."


In [37]:
df_media['media_unido'].isnull().sum()

942

In [38]:
df_media.dropna(subset= 'media_unido', axis = 0, inplace= True)

In [39]:
df_media['media_unido'].isnull().sum()

0

In [40]:
df_languages.to_csv("data/df_languages.csv")
df_viz.to_csv("data/df_viz.csv")
df_code_env.to_csv("data/df_code_env.csv")
df_notebooks.to_csv("data/df_notebooks.csv")
df_learn.to_csv("data/df_learn.csv")
df_share.to_csv("data/df_share.csv")
df_courses.to_csv("data/df_courses.csv")
df_media.to_csv("data/df_media.csv")